In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np

# Plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# ML
from regression import PCovR, KPCovR, SparseKPCovR
from regression import LR, KRR
from kernels import build_kernel, linear_kernel, gaussian_kernel
from kernels import center_kernel, center_kernel_fast
from kernels import center_kernel_oos, center_kernel_oos_fast
from soap import extract_species_pair_groups
from errors import MAE

from sklearn.svm import SVC, LinearSVC
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Utilities
import h5py
import json
import itertools
from tqdm.notebook import tqdm
import project_utils as utils
from tools import load_json, save_json

# Import COSMO style toolkit
import cosmoplot.colorbars as cosmocbars
import cosmoplot.utils as cosmoutils
import cosmoplot.style as cosmostyle

cosmostyle.set_style('article')
colorList = cosmostyle.color_cycle

In /home/helfrech/.config/matplotlib/stylelib/cosmo.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/helfrech/.config/matplotlib/stylelib/cosmoLarge.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/_collections_abc.py:841: MatplotlibDeprecationWarning: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
  self[key] = other[key]


In [3]:
# sys.path.append('/scratch/helfrech/Sync/GDrive/Projects/KPCovR/kernel-tutorials')
# sys.path.append('/scratch/helfrech/Sync/GDrive/Projects/KPCovR/KernelPCovR/analysis/scripts')
# from utilities.sklearn_covr.kpcovr import KernelPCovR as KPCovR2
# from utilities.sklearn_covr.pcovr import PCovR as PCovR2
# from helpers import l_regr, l_kpcovr

# Load train and test splits

In [4]:
# Load SOAP cutoffs
with open('../Processed_Data/soap_hyperparameters.json', 'r') as f:
    soap_hyperparameters = json.load(f)
    
cutoffs = soap_hyperparameters['interaction_cutoff']

In [5]:
# Load train and test set indices for Deem
idxs_deem_train = np.loadtxt('../Processed_Data/DEEM_10k/train.idxs', dtype=int)
idxs_deem_test = np.loadtxt('../Processed_Data/DEEM_10k/test.idxs', dtype=int)

# Total number of structures
n_deem_train = idxs_deem_train.size
n_deem_test = idxs_deem_test.size
n_deem = n_deem_train + n_deem_test

print(n_deem_train, n_deem_test)

7750 2250


In [6]:
# Make dummy DEEM cantons
cantons_deem = np.ones(n_deem, dtype=int) * 4

In [7]:
# Load IZA cantons
cantons_iza = np.loadtxt('../Raw_Data/GULP/IZA_226/cantons.txt', usecols=1, dtype=int)
RWY = np.nonzero(cantons_iza == 4)[0][0]

In [8]:
cantons_iza = np.delete(cantons_iza, RWY)
n_iza = len(cantons_iza)

In [9]:
idxs_iza_train_file = '../Processed_Data/IZA_226/train.idxs' 
idxs_iza_test_file = '../Processed_Data/IZA_226/test.idxs'

# Load IZA train and test set indices
try:
    idxs_iza_train = np.loadtxt(idxs_iza_train_file, dtype=int)
    idxs_iza_test = np.loadtxt(idxs_iza_test_file, dtype=int)
    n_iza_train = len(idxs_iza_train)
    n_iza_test = len(idxs_iza_test)
    
    print(n_iza_train, n_iza_test)

# Compute indices if they don't exist
except IOError:

    # Select IZA sample
    n_iza_train = n_iza // 2
    n_iza_test = n_iza - n_iza_train
    idxs_iza = np.arange(0, n_iza)
    np.random.shuffle(idxs_iza)

    idxs_iza_train = idxs_iza[0:n_iza_train]
    idxs_iza_test = idxs_iza[n_iza_train:]
    
    np.savetxt(idxs_iza_train_file, idxs_iza_train, fmt='%d')
    np.savetxt(idxs_iza_test_file, idxs_iza_test, fmt='%d')

112 113


In [10]:
# Build set of "master" canton labels
cantons_train = {}
cantons_test = {}

cantons_train[4] = np.concatenate((cantons_iza[idxs_iza_train], cantons_deem[idxs_deem_train]))
cantons_test[4] = np.concatenate((cantons_iza[idxs_iza_test], cantons_deem[idxs_deem_test]))

cantons_train[2] = np.concatenate((np.ones(len(idxs_iza_train), dtype=int),
                                   np.ones(len(idxs_deem_train), dtype=int) * 2))
cantons_test[2] = np.concatenate((np.ones(len(idxs_iza_test), dtype=int),
                                  np.ones(len(idxs_deem_test), dtype=int) * 2))

In [11]:
# Dummy Deem cantons to test the "null" case -- i.e., the SVC shouldn't be able to classify
# a random splitting of DEEM
dummy_deem_file_2 = '../Processed_Data/DEEM_10k/dummy_cantons_2-class.dat'
dummy_deem_file_4 = '../Processed_Data/DEEM_10k/dummy_cantons_4-class.dat'

try:
    dummy_cantons_deem_2 = np.loadtxt(dummy_deem_file_2)
    dummy_cantons_deem_4 = np.loadtxt(dummy_deem_file_4)

except IOError:
    dummy_cantons_deem_2 = np.random.randint(1, 3, size=n_deem)
    dummy_cantons_deem_4 = np.random.randint(1, 5, size=n_deem)
    
    np.savetxt(dummy_deem_file_2, dummy_cantons_deem_2)
    np.savetxt(dummy_deem_file_4, dummy_cantons_deem_4)

dummy_cantons_deem_train = {2: dummy_cantons_deem_2[idxs_deem_train],
                            4: dummy_cantons_deem_4[idxs_deem_train]}
dummy_cantons_deem_test = {2: dummy_cantons_deem_2[idxs_deem_test],
                           4: dummy_cantons_deem_4[idxs_deem_test]}

# Model setup

In [12]:
model_dir = '../Processed_Data/Models'

In [13]:
class_names = {}
class_names[2] = ['IZA', 'DEEM']
class_names[4] = ['IZA1', 'IZA2', 'IZA3', 'DEEM']

In [14]:
# Global model parameters
# TODO: or use .get_params()?
# TODO: load from optimization?
# TODO: should we make sure break_ties=True?

# NOTE: with balance weights, the sum of the weights times class frequencies is one
# NOTE: LSVC decision functions are OVR (or Crammer-Singer) only; KSVC decision functions
# are always computed with OVO, but can be converted to OVR. Hence we use OVR with both
# LSVC and KSVC for consistency
svc_kwargs = dict(linear=dict(penalty='l2',
                              loss='squared_hinge',
                              dual=False,
                              multi_class='ovr',
                              #class_weight=None,
                              class_weight='balanced',
                              fit_intercept=True,
                              intercept_scaling=1.0,
                              tol=1.0E-3,
                              max_iter=5000,
                              C=1.0),
                  kernel=dict(kernel='precomputed',
                              decision_function_shape='ovr',
                              #class_weight=None,
                              class_weight='balanced',
                              break_ties=False,
                              tol=1.0E-3,
                              max_iter=-1,
                              C=10.0))

pcovr_kwargs = dict(linear=dict(n_components=None, alpha=0.0, regularization=1.0E-12),
                    kernel=dict(n_components=None, alpha=0.0, regularization=1.0E-12))

In [15]:
# Slices for saving KSVC and KPCovR outputs
deem_train_slice = slice(n_iza_train, None)
deem_test_slice = slice(n_iza_test, None)
iza_train_slice = slice(0, n_iza_train)
iza_test_slice = slice(0, n_iza_test)

# Build kernels

In [16]:
# Load the kernels or compute if they don't exist
for cutoff in cutoffs:
    for kernel_type in ('linear', 'gaussian'):
        kernel_name = kernel_type.capitalize()
        
        work_dir = f'{model_dir}/{cutoff}/Kernel_Models/{kernel_name}/KSVC-KPCovR'
        
        if not os.path.exists(work_dir):
            os.makedirs(work_dir)
        
        # File to store kernels for re-use
        kernel_file = f'{work_dir}/structure_kernels.hdf5'
        kernel_parameter_file = f'{work_dir}/volumes_mae_parameters.json'

        if not os.path.exists(kernel_file):
            
            # SOAP files (atomwise, FPS'ed features)
            deem_file = f'{deem_dir}/{cutoff}/soaps.hdf5'
            iza_file = f'{iza_dir}/{cutoff}/soaps.hdf5'

            # Assemble the train and test set SOAPs from IZA and DEEM
            soaps_train, soaps_test = utils.load_soaps(deem_file, iza_file,
                                                       idxs_deem_train, idxs_deem_test,
                                                       idxs_iza_train, idxs_iza_test,
                                                       idxs_iza_delete=[RWY])

            # Compute kernels
            # This can be consolidated if doing linear KRR
            if kernel_type == 'gaussian':
                kernel_parameters = load_json(kernel_parameter_file)
                kernel_parameters.pop('sigma')
                kernel_parameters.pop('regularization')
            else:
                kernel_parameters = dict(kernel='linear', zeta=1)
            
            utils.compute_kernels(soaps_train, soaps_test, **kernel_parameters, kernel_file=kernel_file)

# Kernel models

In [17]:
deem_name = 'DEEM_10k'
iza_name = 'IZA_226onDEEM_10k'
deem_dir = f'../Processed_Data/{deem_name}/Data'
iza_dir = f'../Processed_Data/{iza_name}/Data'

## KernelSVC

In [18]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    
    for kernel_type in tqdm(('linear', 'gaussian'), desc='Kernel', leave=False):
        kernel_name = kernel_type.capitalize()
        
        kernel_dir = f'{model_dir}/{cutoff}/Kernel_Models/{kernel_name}/KSVC-KPCovR'
        
        # File to store kernels for re-use
        kernel_file = f'{kernel_dir}/structure_kernels.hdf5'
        
        # Load kernels
        K_train, K_test, K_test_test = utils.load_kernels(kernel_file)

        # Center and scale kernels
        K_train, K_test, K_test_test = \
            utils.preprocess_kernels(K_train, K_test=K_test, 
                                     K_test_test=K_test_test, K_bridge=K_test)
        
        for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
            
            # Prepare outputs
            output_dir = f'Kernel_Models/{kernel_name}/KSVC-KPCovR/{n_cantons}-Class'
            
            if not os.path.exists(f'{deem_dir}/{cutoff}/{output_dir}'):
                os.makedirs(f'{deem_dir}/{cutoff}/{output_dir}')
                
            if not os.path.exists(f'{iza_dir}/{cutoff}/{output_dir}'):
                os.makedirs(f'{iza_dir}/{cutoff}/{output_dir}')
            
            svc_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'
            svc_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'
            
            svc_cantons_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/svc_structure_cantons.dat'
            svc_cantons_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/svc_structure_cantons.dat'
            
            parameter_dir = f'{kernel_dir}/{n_cantons}-Class'
            svc_parameter_file = f'{parameter_dir}/svc_parameters.json'
            svc_model_file = f'{parameter_dir}/svc.json'

            # Run KSVC
            svc_parameters = svc_kwargs['kernel'].copy() #load_json(svc_parameter_file) ###
            df_train, df_test, predicted_cantons_train, predicted_cantons_test = \
                utils.do_svc(K_train, K_test, cantons_train[n_cantons], cantons_test[n_cantons], 
                             svc_type='kernel', **svc_parameters,
                             outputs=['decision_functions', 'predictions'],
                             save_model=svc_model_file)
            
            # Save IZA and DEEM KSVC decision functions
            utils.split_and_save(df_train, df_test,
                           idxs_deem_train, idxs_deem_test,
                           deem_train_slice, deem_test_slice,
                           output=svc_df_deem_file, output_format='%f',
                           hdf5_attrs=None)                           
            
            utils.split_and_save(df_train, df_test,
                                 idxs_iza_train, idxs_iza_test,
                                 iza_train_slice, iza_test_slice,
                                 output=svc_df_iza_file, output_format='%f',
                                 hdf5_attrs=None)
            
            # Save IZA and DEEM KSVC canton predictions
            utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                 idxs_deem_train, idxs_deem_test,
                                 deem_train_slice, deem_test_slice,
                                 output=svc_cantons_deem_file, output_format='%d',
                                 hdf5_attrs=None)
            
            utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                 idxs_iza_train, idxs_iza_test,
                                 iza_train_slice, iza_test_slice,
                                 output=svc_cantons_iza_file, output_format='%d',
                                 hdf5_attrs=None)

## Check that a random split of DEEM can't be predicted

In [18]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    
    for kernel_type in tqdm(('linear', 'gaussian'), desc='Kernel', leave=False):
        kernel_name = kernel_type.capitalize()
        
        kernel_dir = f'{model_dir}/{cutoff}/Kernel_Models/{kernel_name}/KSVC-KPCovR'
        
        # File to store kernels for re-use
        kernel_file = f'{kernel_dir}/structure_kernels.hdf5'
        
        # Load kernels
        K_train, K_test, K_test_test = utils.load_kernels(kernel_file)
        K_train = K_train[deem_train_slice, :][:, deem_train_slice]
        K_test = K_test[deem_test_slice, :][:, deem_train_slice]
        K_test_test = K_test_test[deem_test_slice, :][:, deem_test_slice]
        
        # Center and scale kernels
        K_train, K_test, K_test_test = \
            utils.preprocess_kernels(K_train, K_test=K_test, 
                                     K_test_test=K_test_test, K_bridge=K_test)
        
        for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
            
            # Prepare outputs
            output_dir = f'Kernel_Models/{kernel_name}/KSVC-KPCovR/{n_cantons}-Class'
            
            svc_cantons_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/dummy_svc_structure_cantons.dat'
                        
            parameter_dir = f'{kernel_dir}/{n_cantons}-Class'
            svc_parameter_file = f'{parameter_dir}/svc_parameters.json'

            # Run KSVC
            svc_parameters = svc_kwargs['kernel'].copy() #load_json(svc_parameter_file) ###
            #svc_parameters['class_weight'] = None ###
            predicted_cantons_train, predicted_cantons_test = \
                utils.do_svc(K_train, K_test, 
                             dummy_cantons_deem_train[n_cantons], 
                             dummy_cantons_deem_test[n_cantons], 
                             svc_type='kernel', **svc_parameters,
                             outputs=['predictions'])
            
            # Save DEEM KSVC canton predictions
            utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                 idxs_deem_train, idxs_deem_test,
                                 slice(None), slice(None),
                                 output=svc_cantons_deem_file, output_format='%d',
                                 hdf5_attrs=None)
            
            print(accuracy_score(dummy_cantons_deem_train[n_cantons], predicted_cantons_train))
            print(confusion_matrix(dummy_cantons_deem_train[n_cantons], predicted_cantons_train))
            print(accuracy_score(dummy_cantons_deem_test[n_cantons], predicted_cantons_test))
            print(confusion_matrix(dummy_cantons_deem_test[n_cantons], predicted_cantons_test))

0.528258064516129
[[1865 2029]
 [1627 2229]]
0.496
[[510 625]
 [509 606]]
0.3
[[313 416 466 770]
 [200 529 469 769]
 [230 402 595 750]
 [215 350 388 888]]
0.2528888888888889
[[ 84 122 151 238]
 [ 80  97 159 225]
 [ 71 105 132 217]
 [ 70 114 129 256]]


0.5655483870967742
[[2027 1867]
 [1500 2356]]
0.492
[[495 640]
 [503 612]]
0.33548387096774196
[[469 386 395 715]
 [279 580 418 690]
 [275 338 664 700]
 [264 327 363 887]]
0.25466666666666665
[[105 123 143 224]
 [ 99 114 124 224]
 [ 79 100 125 221]
 [ 89 128 123 229]]


0.5606451612903226
[[2146 1748]
 [1657 2199]]
0.5168888888888888
[[580 555]
 [532 583]]
0.3481290322580645
[[685 368 381 531]
 [423 600 380 564]
 [411 389 628 549]
 [382 343 331 785]]
0.24755555555555556
[[155 118 139 183]
 [136 109 139 177]
 [116 121 125 163]
 [153 146 102 168]]


0.7296774193548388
[[2819 1075]
 [1020 2836]]
0.5071111111111111
[[559 576]
 [533 582]]
0.5655483870967742
[[1093  281  253  338]
 [ 275 1111  235  346]
 [ 301  286 1080  310]
 [ 247  274  221 1099]]
0.23822222222222222
[[137 145 155 158]
 [153 133 138 137]
 [142 124 118 141]
 [151 131 139 148]]



## KRR check

In [ ]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    
    for kernel_type in tqdm(('linear', 'gaussian'), desc='Kernel', leave=False):
        kernel_name = kernel_type.capitalize()
        
        kernel_dir = f'{model_dir}/{cutoff}/Kernel_Models/{kernel_name}/KSVC-KPCovR'
        
        # Load kernels
        kernel_file = f'{kernel_dir}/structure_kernels.hdf5'
        K_train, K_test, K_test_test = utils.load_kernels(kernel_file)

        # Center and scale kernels
        K_train, K_test, K_test_test = \
            utils.preprocess_kernels(K_train, K_test=K_test, 
                                     K_test_test=K_test_test, K_bridge=K_test)
        
        for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
            
            # Load decision functions
            input_dir = f'Kernel_Models/{kernel_name}/KSVC-KPCovR/{n_cantons}-Class'
            svc_df_deem_file = f'{deem_dir}/{cutoff}/{input_dir}/svc_structure_dfs.dat'
            svc_df_iza_file = f'{iza_dir}/{cutoff}/{input_dir}/svc_structure_dfs.dat'
            
            df_train, df_test = utils.load_data(svc_df_deem_file, svc_df_iza_file,
                                                idxs_deem_train, idxs_deem_test,
                                                idxs_iza_train, idxs_iza_test)
            
            # Center and scale the decision functions
            df_train, df_test, df_center, df_scale = \
                utils.preprocess_data(df_train, df_test)

            # Check that KRR can reproduce the decision functions
            dfp_train, dfp_test = utils.regression_check(K_train, K_test, 
                                                         df_train, df_test, 
                                                         regression_type='kernel')
            
            print(MAE(dfp_train, df_train))
            print(MAE(dfp_test, df_test))

## KPCovR

In [20]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    
    for kernel_type in tqdm(('linear', 'gaussian'), desc='Kernel', leave=False):
        kernel_name = kernel_type.capitalize()
        
        kernel_dir = f'{model_dir}/{cutoff}/Kernel_Models/{kernel_name}/KSVC-KPCovR'
        
        # Load kernels
        kernel_file = f'{kernel_dir}/structure_kernels.hdf5'
        K_train, K_test, K_test_test = utils.load_kernels(kernel_file)

        # Center and scale kernels
        K_train, K_test, K_test_test = \
            utils.preprocess_kernels(K_train, K_test=K_test, 
                                     K_test_test=K_test_test, K_bridge=K_test)
        
        for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
            
            # Prepare outputs
            output_dir = f'Kernel_Models/{kernel_name}/KSVC-KPCovR/{n_cantons}-Class'
            
            svc_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'
            svc_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'
            
            pcovr_projection_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/pcovr_structures.hdf5' 
            pcovr_projection_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/pcovr_structures.hdf5'
            
            pcovr_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/pcovr_structure_dfs.dat'
            pcovr_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/pcovr_structure_dfs.dat'
            
            pcovr_cantons_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/pcovr_structure_cantons.dat'
            pcovr_cantons_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/pcovr_structure_cantons.dat'
            
            parameter_dir = f'{kernel_dir}/{n_cantons}-Class'
            pcovr_parameter_file = f'{parameter_dir}/pcovr_parameters.json'
            pcovr_model_file = f'{parameter_dir}/pcovr.json'
            svc_parameter_file = f'{parameter_dir}/svc_parameters.json'
            
            df_train, df_test = utils.load_data(svc_df_deem_file, svc_df_iza_file,
                                                idxs_deem_train, idxs_deem_test,
                                                idxs_iza_train, idxs_iza_test)
            
            # Center and scale the decision functions
            df_train, df_test, df_center, df_scale = \
                utils.preprocess_data(df_train, df_test)
            
            # Run KPCovR
            pcovr_parameters = pcovr_kwargs['kernel'].copy() #load_json(pcovr_parameter_file) ###
            svc_parameters = svc_kwargs['kernel'].copy() #load_json(svc_parameter_file) ###
            T_train, T_test, dfp_train, dfp_test = \
                utils.do_pcovr(K_train, K_test, df_train, df_test, 
                               pcovr_type='kernel', **pcovr_parameters,
                               save_model=pcovr_model_file)
                      
            # Post process the KPCovR decision functions
            # (i.e., turn them back into canton predictions)
            predicted_cantons_train, predicted_cantons_test = \
                utils.postprocess_decision_functions(dfp_train, dfp_test, df_center, df_scale,
                                                     df_type=svc_parameters['decision_function_shape'],
                                                     n_classes=n_cantons)
            
            # Save IZA and DEEM KPCovR projections
            utils.split_and_save(T_train, T_test,
                                 idxs_deem_train, idxs_deem_test,
                                 deem_train_slice, deem_test_slice,
                                 output=pcovr_projection_deem_file, output_format='%f',
                                 hdf5_attrs=pcovr_parameters)
            
            utils.split_and_save(T_train, T_test,
                                 idxs_iza_train, idxs_iza_test,
                                 iza_train_slice, iza_test_slice,
                                 output=pcovr_projection_iza_file, output_format='%f',
                                 hdf5_attrs=pcovr_parameters)            
            
            # Save IZA and DEEM KPCovR decision functions
            utils.split_and_save(dfp_train, dfp_test,
                                 idxs_deem_train, idxs_deem_test,
                                 deem_train_slice, deem_test_slice,
                                 output=pcovr_df_deem_file, output_format='%f',
                                 hdf5_attrs=None)
            
            utils.split_and_save(dfp_train, dfp_test,
                                 idxs_iza_train, idxs_iza_test,
                                 iza_train_slice, iza_test_slice,
                                 output=pcovr_df_iza_file, output_format='%f', 
                                 hdf5_attrs=None)
            
            # Save IZA and DEEM KPCovR canton predictions
            utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                 idxs_deem_train, idxs_deem_test,
                                 deem_train_slice, deem_test_slice,
                                 output=pcovr_cantons_deem_file, output_format='%d',
                                 hdf5_attrs=None)
            
            utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                 idxs_iza_train, idxs_iza_test,
                                 iza_train_slice, iza_test_slice,
                                 output=pcovr_cantons_iza_file, output_format='%d',
                                 hdf5_attrs=None)

# Linear Models

In [21]:
# Linear model setup
n_species = 2
group_names = {'power': ['OO', 'OSi', 'SiSi', 
                         'OO+OSi', 'OO+SiSi', 'OSi+SiSi',
                         'OO+OSi+SiSi'], 
               'radial': ['O', 'Si', 'O+Si']}

In [22]:
deem_name = 'DEEM_10k'
all_deem_name = 'DEEM_330k'
iza_name = 'IZA_226'
deem_dir = f'../Processed_Data/{deem_name}/Data'
all_deem_dir = f'../Processed_Data/{all_deem_name}/Data'
iza_dir = f'../Processed_Data/{iza_name}/Data'

## LinearSVC

In [25]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    linear_dir = f'{model_dir}/{cutoff}/Linear_Models/LSVC-LPCovR'
    
    for spectrum_type in tqdm(('power', 'radial'), desc='Spectrum', leave=False):
        spectrum_name = spectrum_type.capitalize()
        
        # Load SOAPs
        deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        iza_file = f'{iza_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        
        soaps_train, soaps_test = utils.load_soaps(deem_file, iza_file,
                                                   idxs_deem_train, idxs_deem_test,
                                                   idxs_iza_train, idxs_iza_test,
                                                   idxs_iza_delete=[RWY],
                                                   train_test_concatenate=True)

        # Scale the SOAPs so they are of a 'usable' magnitude for the SVC
        soaps_train, soaps_test, soap_scale = \
            utils.preprocess_soaps(soaps_train, soaps_test, return_scale=True)
        
        n_features = soaps_train.shape[1]
        feature_groups = extract_species_pair_groups(n_features, n_species, 
                                                     spectrum_type=spectrum_type,
                                                     combinations=True)
        # Prepare loading of the DEEM 330k structures 
        all_deem_file = f'{all_deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        f = h5py.File(all_deem_file, 'r')
        deem_330k_dataset = f['0']

        for species_pairing, feature_idxs in zip(tqdm(group_names[spectrum_type], 
                                                      desc='Species', leave=False),
                                                 feature_groups):

            for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
                                
                # Prepare outputs
                output_dir = f'Linear_Models/LSVC-LPCovR/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                
                if not os.path.exists(f'{deem_dir}/{cutoff}/{output_dir}'):
                    os.makedirs(f'{deem_dir}/{cutoff}/{output_dir}')
                    
                if not os.path.exists(f'{all_deem_dir}/{cutoff}/{output_dir}'):
                    os.makedirs(f'{all_deem_dir}/{cutoff}/{output_dir}')
                    
                if not os.path.exists(f'{iza_dir}/{cutoff}/{output_dir}'):
                    os.makedirs(f'{iza_dir}/{cutoff}/{output_dir}')
                
                svc_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'
                svc_df_all_deem_file = f'{all_deem_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'
                svc_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'

                svc_cantons_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/svc_structure_cantons.dat'
                svc_cantons_all_deem_file = f'{all_deem_dir}/{cutoff}/{output_dir}/svc_structure_cantons.dat'
                svc_cantons_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/svc_structure_cantons.dat'
                                
                parameter_dir = f'{linear_dir}/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                svc_parameter_file = f'{parameter_dir}/svc_parameters.json'
                svc_model_file = f'{parameter_dir}/svc.json'
                
                svc_weights_file = f'{parameter_dir}/svc_weights.dat'

                # Run LSVC
                svc_parameters = svc_kwargs['linear'].copy() #load_json(svc_parameter_file) ###
                df_train, df_test, predicted_cantons_train, predicted_cantons_test, weights = \
                    utils.do_svc(soaps_train[:, feature_idxs], soaps_test[:, feature_idxs], 
                                 cantons_train[n_cantons], cantons_test[n_cantons], 
                                 svc_type='linear', **svc_parameters,
                                 outputs=['decision_functions', 'predictions', 'weights'],
                                 save_model=svc_model_file)

                # Save IZA and DEEM LSVC decision functions
                utils.split_and_save(df_train, df_test,
                                     idxs_deem_train, idxs_deem_test,
                                     deem_train_slice, deem_test_slice,
                                     output=svc_df_deem_file, output_format='%f',
                                     hdf5_attrs=None)
                
                utils.split_and_save(df_train, df_test,
                                     idxs_iza_train, idxs_iza_test,
                                     iza_train_slice, iza_test_slice,
                                     output=svc_df_iza_file, output_format='%f',
                                     hdf5_attrs=None)

                # Save IZA and DEEM LSVC canton predictions
                utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                     idxs_deem_train, idxs_deem_test,
                                     deem_train_slice, deem_test_slice,
                                     output=svc_cantons_deem_file, output_format='%d',
                                     hdf5_attrs=None)
                
                utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                     idxs_iza_train, idxs_iza_test,
                                     iza_train_slice, iza_test_slice,
                                     output=svc_cantons_iza_file, output_format='%d',
                                     hdf5_attrs=None)
                
                # Save weights
                # TODO: when constructing the real space density, 
                # do we also have to account for the intercept?
                np.savetxt(svc_weights_file, weights)
                
                # Load the SVC model
                svc_model_dict = load_json(svc_model_file, array_convert=True)
                svc_model = LinearSVC()
                svc_model.__dict__ = svc_model_dict
                
                # Read the DEEM_330k structures and compute decision functions
                # and canton predictions
                
                # We could load up the whole dataset beforehand and probably make this faster
                # since we wouldn't have to read the HDF5 for both the DFs and predictions,
                # but the single-read seems to be memory-prohibitive and this 
                # multiple-read construction appears tolerably fast
                # and also seems to use much less memory
                df_all_deem = svc_model.decision_function(deem_330k_dataset[:, feature_idxs]/soap_scale)
                predicted_cantons_all_deem = svc_model.predict(deem_330k_dataset[:, feature_idxs]/soap_scale)
                
                # Save DEEM 330k LSVC decision functions and canton predictions
                np.savetxt(svc_df_all_deem_file, df_all_deem)
                np.savetxt(svc_cantons_all_deem_file, predicted_cantons_all_deem, fmt='%d')
                
        f.close()

## Check that a random split of DEEM can't be predicted

In [25]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    linear_dir = f'{model_dir}/{cutoff}/Linear_Models/LSVC-LPCovR'
    
    for spectrum_type in tqdm(('power', 'radial'), desc='Spectrum', leave=False):
        spectrum_name = spectrum_type.capitalize()
        
        # Load SOAPs
        deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        iza_file = f'{iza_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        
        soaps_train, soaps_test = utils.load_soaps(deem_file, iza_file,
                                                   idxs_deem_train, idxs_deem_test,
                                                   idxs_iza_train, idxs_iza_test,
                                                   idxs_iza_delete=[RWY],
                                                   train_test_concatenate=True)
        
        soaps_train = soaps_train[deem_train_slice]
        soaps_test = soaps_test[deem_test_slice]

        # Scale the SOAPs so they are of a 'usable' magnitude for the SVC
        soaps_train, soaps_test = utils.preprocess_soaps(soaps_train, soaps_test)
        
        n_features = soaps_train.shape[1]
        feature_groups = extract_species_pair_groups(n_features, n_species, 
                                                     spectrum_type=spectrum_type,
                                                     combinations=True)

        for species_pairing, feature_idxs in zip(tqdm(group_names[spectrum_type], 
                                                      desc='Species', leave=False),
                                                 feature_groups):
            
            for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
                
                # Prepare outputs
                output_dir = f'Linear_Models/LSVC-LPCovR/{n_cantons}-Class/{spectrum_name}/{species_pairing}'

                svc_cantons_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/dummy_svc_structure_cantons.dat'
                                
                parameter_dir = f'{linear_dir}/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                svc_parameter_file = f'{parameter_dir}/svc_parameters.json'
                
                svc_weights_file = f'{parameter_dir}/svc_weights.dat'

                # Run LSVC
                svc_parameters = svc_kwargs['linear'].copy() #load_json(svc_parameter_file) ###
                #svc_parameters['class_weight'] = None ###
                predicted_cantons_train, predicted_cantons_test = \
                    utils.do_svc(soaps_train[:, feature_idxs], soaps_test[:, feature_idxs], 
                                 dummy_cantons_deem_train[n_cantons], 
                                 dummy_cantons_deem_test[n_cantons], 
                                 svc_type='linear', **svc_parameters,
                                 outputs=['predictions'])
                
                # Save DEEM LSVC canton predictions
                utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                     idxs_deem_train, idxs_deem_test,
                                     slice(None), slice(None),
                                     output=svc_cantons_deem_file, output_format='%d',
                                     hdf5_attrs=None)
                
                print(accuracy_score(dummy_cantons_deem_train[n_cantons], predicted_cantons_train))
                print(confusion_matrix(dummy_cantons_deem_train[n_cantons], predicted_cantons_train))
                print(accuracy_score(dummy_cantons_deem_test[n_cantons], predicted_cantons_test))
                print(confusion_matrix(dummy_cantons_deem_test[n_cantons], predicted_cantons_test))

0.520774193548387
[[1858 2036]
 [1678 2178]]
0.5022222222222222
[[527 608]
 [512 603]]
0.2713548387096774
[[343 543 554 525]
 [285 592 547 543]
 [275 557 588 557]
 [260 519 482 580]]
0.2408888888888889
[[ 92 170 168 165]
 [ 80 139 182 160]
 [ 82 143 152 148]
 [ 95 165 150 159]]


0.535483870967742
[[2005 1889]
 [1711 2145]]
0.5017777777777778
[[525 610]
 [511 604]]
0.2663225806451613
[[246 560 545 614]
 [243 556 546 622]
 [241 538 577 621]
 [223 430 503 685]]
0.2457777777777778
[[ 72 178 166 179]
 [ 74 133 172 182]
 [ 65 143 166 151]
 [ 69 143 175 182]]


0.5163870967741936
[[1863 2031]
 [1717 2139]]
0.49866666666666665
[[519 616]
 [512 603]]
0.26670967741935486
[[424 600 398 543]
 [380 614 449 524]
 [416 580 475 506]
 [369 545 373 554]]
0.24
[[124 193 132 146]
 [107 151 141 162]
 [130 154 108 133]
 [106 181 125 157]]


0.5508387096774193
[[2048 1846]
 [1635 2221]]
0.49333333333333335
[[509 626]
 [514 601]]
0.28735483870967743
[[203 620 489 653]
 [151 702 466 648]
 [165 596 560 656]
 [135 503 441 762]]
0.24666666666666667
[[ 51 180 159 205]
 [ 46 163 164 188]
 [ 53 143 139 190]
 [ 56 159 152 202]]


0.5369032258064517
[[2002 1892]
 [1697 2159]]
0.5057777777777778
[[536 599]
 [513 602]]
0.2718709677419355
[[332 641 372 620]
 [290 681 399 597]
 [329 606 424 618]
 [273 545 353 670]]
0.23777777777777778
[[ 95 191 118 191]
 [ 92 159 126 184]
 [102 151 102 170]
 [ 88 189 113 179]]


...

0.6068387096774194
[[2375 1519]
 [1528 2328]]
0.5048888888888889
[[561 574]
 [540 575]]
0.38167741935483873
[[648 397 474 446]
 [367 699 443 458]
 [357 364 856 400]
 [325 334 427 755]]
0.24
[[124 139 184 148]
 [117 122 153 169]
 [105 130 148 142]
 [126 141 156 146]]


0.6268387096774194
[[2458 1436]
 [1456 2400]]
0.5022222222222222
[[557 578]
 [542 573]]
0.40825806451612906
[[714 407 419 425]
 [353 835 345 434]
 [384 397 779 417]
 [316 368 321 836]]
0.24044444444444443
[[123 156 157 159]
 [120 144 135 162]
 [115 135 123 152]
 [124 169 125 151]]


0.5073548387096775
[[2114 1780]
 [2038 1818]]
0.5044444444444445
[[637 498]
 [617 498]]
0.2603870967741935
[[438 460 422 645]
 [438 468 411 650]
 [443 429 453 652]
 [396 400 386 659]]
0.25066666666666665
[[143 144 123 185]
 [124 112 127 198]
 [111 117 115 182]
 [139 129 107 194]]


0.5077419354838709
[[2141 1753]
 [2062 1794]]
0.5146666666666667
[[656 479]
 [613 502]]
0.26464516129032256
[[396 577 366 626]
 [356 602 401 608]
 [381 576 417 603]
 [364 493 348 636]]
0.25466666666666665
[[124 190 120 161]
 [107 157 113 184]
 [110 147  99 169]
 [110 177  89 193]]


0.5092903225806452
[[2130 1764]
 [2039 1817]]
0.4968888888888889
[[630 505]
 [627 488]]
0.2709677419354839
[[510 479 339 637]
 [426 517 367 657]
 [459 479 365 674]
 [407 432 294 708]]
0.24977777777777777
[[141 160  94 200]
 [119 135 104 203]
 [133 124  82 186]
 [140 148  77 204]]



## LR check

In [ ]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    
    for spectrum_type in tqdm(('power', 'radial'), desc='Spectrum', leave=False):
        spectrum_name = spectrum_type.capitalize()
        
        # Load SOAPs
        deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        iza_file = f'{iza_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'

        soaps_train, soaps_test = utils.load_soaps(deem_file, iza_file,
                                                   idxs_deem_train, idxs_deem_test,
                                                   idxs_iza_train, idxs_iza_test,
                                                   idxs_iza_delete=[RWY],
                                                   train_test_concatenate=True)
        
        n_features = soaps_train.shape[1]
        feature_groups = extract_species_pair_groups(n_features, n_species, 
                                                     spectrum_type=spectrum_type, 
                                                     combinations=True)
        
        for species_pairing, feature_idxs in zip(tqdm(group_names[spectrum_type], 
                                                      desc='Species', leave=False),
                                                 feature_groups):
            
            x_train = soaps_train[:, feature_idxs]
            x_test = soaps_test[:, feature_idxs]

            # Preprocess the SOAPs like the decision functions
            # (i.e., center and scale) for the regression.
            x_train, x_test, x_center, x_scale = \
                utils.preprocess_data(x_train, x_test)
            
            for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
                
                # Load decision functions
                input_dir = f'Linear_Models/LSVC-LPCovR/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                svc_df_deem_file = f'{deem_dir}/{cutoff}/{input_dir}/svc_structure_dfs.dat'
                svc_df_iza_file = f'{iza_dir}/{cutoff}/{input_dir}/svc_structure_dfs.dat'

                df_train, df_test = utils.load_data(svc_df_deem_file, svc_df_iza_file,
                                                    idxs_deem_train, idxs_deem_test,
                                                    idxs_iza_train, idxs_iza_test)
                
                # Center and scale the decision functions
                df_train, df_test, df_center, df_scale = \
                    utils.preprocess_data(df_train, df_test)
                
                # Check that LR can reproduce the decision functions
                dfp_train, dfp_test = utils.regression_check(x_train, x_test, 
                                                             df_train, df_test, 
                                                             regression_type='linear')
                
                print(MAE(dfp_train, df_train))
                print(MAE(dfp_test, df_test))

## PCovR

In [28]:
batch_size = 100000

In [33]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    linear_dir = f'{model_dir}/{cutoff}/Linear_Models/LSVC-LPCovR'
    
    for spectrum_type in tqdm(('power', 'radial'), desc='Spectrum', leave=False):
        spectrum_name = spectrum_type.capitalize()
        
        # Load SOAPs
        deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        iza_file = f'{iza_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'

        soaps_train, soaps_test = utils.load_soaps(deem_file, iza_file,
                                                   idxs_deem_train, idxs_deem_test,
                                                   idxs_iza_train, idxs_iza_test,
                                                   idxs_iza_delete=[RWY],
                                                   train_test_concatenate=True)
        
        # Scale the SOAPs so they are of a 'usable' magnitude for the SVC
        #soaps_train, soaps_test = utils.preprocess_soaps(soaps_train, soaps_test)
        
        n_features = soaps_train.shape[1]
        feature_groups = extract_species_pair_groups(n_features, n_species, 
                                                     spectrum_type=spectrum_type,
                                                     combinations=True)
        
        # Prepare loading of the DEEM 330k structures
        all_deem_file = f'{all_deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        f = h5py.File(all_deem_file, 'r')
        deem_330k_dataset = f['0']
        
        # Prepare batches for PCovR
        n_samples_330k = deem_330k_dataset.len()
        n_batches = n_samples_330k // batch_size
        if n_samples_330k % batch_size > 0:
            n_batches += 1
        
        for species_pairing, feature_idxs in zip(tqdm(group_names[spectrum_type], 
                                                      desc='Species', leave=False),
                                                 feature_groups):
            
            x_train = soaps_train[:, feature_idxs]
            x_test = soaps_test[:, feature_idxs]

            # Preprocess the SOAPs like the decision functions
            # (i.e., center and scale) for the regression.
            x_train, x_test, x_center, x_scale = \
                utils.preprocess_data(x_train, x_test)
            
            for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
                
                # Prepare outputs
                output_dir = f'Linear_Models/LSVC-LPCovR/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                
                svc_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'
                svc_df_all_deem_file = f'{all_deem_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat' # Don't actually need
                svc_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/svc_structure_dfs.dat'
                
                pcovr_projection_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/pcovr_structures.hdf5'
                pcovr_projection_all_deem_file = f'{all_deem_dir}/{cutoff}/{output_dir}/pcovr_structures.hdf5'
                pcovr_projection_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/pcovr_structures.hdf5'
                
                pcovr_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/pcovr_structure_dfs.dat'
                pcovr_df_all_deem_file = f'{all_deem_dir}/{cutoff}/{output_dir}/pcovr_structure_dfs.dat'
                pcovr_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/pcovr_structure_dfs.dat'
                
                pcovr_cantons_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/pcovr_structure_cantons.dat'
                pcovr_cantons_all_deem_file = f'{all_deem_dir}/{cutoff}/{output_dir}/pcovr_structure_cantons.dat'
                pcovr_cantons_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/pcovr_structure_cantons.dat'
                
                parameter_dir = f'{linear_dir}/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                pcovr_parameter_file = f'{parameter_dir}/pcovr_parameters.json'
                pcovr_model_file = f'{parameter_dir}/pcovr.json'
                
                # Save the centering and scaling factors
                center_scale_file = f'{parameter_dir}/center_scale.json'
                save_json(dict(center=x_center, scale=x_scale), center_scale_file, array_convert=True)
                
                df_train, df_test = utils.load_data(svc_df_deem_file, svc_df_iza_file,
                                                    idxs_deem_train, idxs_deem_test,
                                                    idxs_iza_train, idxs_iza_test)
            
                # Center and scale the decision functions
                df_train, df_test, df_center, df_scale = \
                    utils.preprocess_data(df_train, df_test)
                
                # Run PCovR
                pcovr_parameters = pcovr_kwargs['linear'].copy() #load_json(pcovr_parameter_file) ###
                svc_parameters = svc_kwargs['linear'].copy() #load_json(svc_parameter_file) ###
                T_train, T_test, dfp_train, dfp_test = \
                    utils.do_pcovr(x_train, x_test, df_train, df_test, 
                                   pcovr_type='linear', **pcovr_parameters,
                                   save_model=pcovr_model_file)

                # Post process the PCovR decision functions
                # (i.e., turn them back into canton predictions)
                predicted_cantons_train, predicted_cantons_test = \
                    utils.postprocess_decision_functions(dfp_train, dfp_test, df_center, df_scale,
                                                         df_type=svc_parameters['multi_class'],
                                                         n_classes=n_cantons)
                
                # Save IZA and DEEM PCovR projections
                utils.split_and_save(T_train, T_test,
                                     idxs_deem_train, idxs_deem_test,
                                     deem_train_slice, deem_test_slice,
                                     output=pcovr_projection_deem_file, output_format='%f',
                                     hdf5_attrs=pcovr_parameters)
                
                utils.split_and_save(T_train, T_test,
                                     idxs_iza_train, idxs_iza_test,
                                     iza_train_slice, iza_test_slice,
                                     output=pcovr_projection_iza_file, output_format='%f',
                                     hdf5_attrs=pcovr_parameters)

                # Save IZA and DEEM PCovR decision functions
                utils.split_and_save(dfp_train, dfp_test,
                                     idxs_deem_train, idxs_deem_test,
                                     deem_train_slice, deem_test_slice,
                                     output=pcovr_df_deem_file, output_format='%f',
                                     hdf5_attrs=None)
                
                utils.split_and_save(dfp_train, dfp_test,
                                     idxs_iza_train, idxs_iza_test,
                                     iza_train_slice, iza_test_slice,
                                     output=pcovr_df_iza_file, output_format='%f',
                                     hdf5_attrs=None)

                # Save IZA and DEEM PCovR canton predictions
                utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                     idxs_deem_train, idxs_deem_test,
                                     deem_train_slice, deem_test_slice,
                                     output=pcovr_cantons_deem_file, output_format='%d',
                                     hdf5_attrs=None)
                
                utils.split_and_save(predicted_cantons_train, predicted_cantons_test,
                                     idxs_iza_train, idxs_iza_test,
                                     iza_train_slice, iza_test_slice,
                                     output=pcovr_cantons_iza_file, output_format='%d',
                                     hdf5_attrs=None)
                
                # Load PCovR model
                pcovr_model_dict = load_json(pcovr_model_file, array_convert=True)
                pcovr_model = PCovR()
                pcovr_model.__dict__ = pcovr_model_dict
                
                # Prepare output arrays for batch processing
                T_all_deem = np.zeros((n_samples_330k, pcovr_model.Pxt.shape[1]))
                df_all_deem = np.zeros((n_samples_330k, pcovr_model.Pty.shape[1]))
                
                # Read the DEEM_330k structures and compute decision functions
                # and canton predictions in batches
                for i in tqdm(range(0, n_batches), desc='Batch', leave=False):
                    batch_slice = slice(i * batch_size, (i + 1) * batch_size)
                    
                    deem_330k_batch = deem_330k_dataset[batch_slice, feature_idxs]
                    deem_330k_batch = (deem_330k_batch - x_center) / x_scale
                    
                    T_all_deem[batch_slice] = pcovr_model.transform(deem_330k_batch)
                    df_all_deem[batch_slice] = pcovr_model.predict(deem_330k_batch)
                  
                # Squeeze the decision functions (required for df_to_class in the 2-class case)
                df_all_deem = np.squeeze(df_all_deem)
                predicted_cantons_all_deem = \
                    utils.df_to_class(df_all_deem * df_scale + df_center, 
                                      df_type=svc_parameters['multi_class'], 
                                      n_classes=n_cantons)
                
                # Save DEEM 330k LPCovR projections, decision functions, and canton predictions
                
                # To save the projections in the same format as `split_and_save`, 
                # use the general HDF5 saving function with list of arrays instead of 2D array
                utils.save_hdf5(pcovr_projection_all_deem_file, list(T_all_deem), attrs=pcovr_parameters)
                np.savetxt(pcovr_df_all_deem_file, df_all_deem, fmt='%f')
                np.savetxt(pcovr_cantons_all_deem_file, predicted_cantons_all_deem, fmt='%d')

# Logistic Regression